In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

sys.path.append("./../")
import os
import sys
from copy import deepcopy

from babisteps.basemodels.nodes import Coordinate, Entity

sys.path.append("./../")
from pathlib import Path

import networkx as nx
import numpy as np
import yaml
from matplotlib import pyplot as plt

from babisteps.basemodels.generators import DELIM
from babisteps.basemodels.nodes import Entity
from babisteps.basemodels.temporaltracking import EntitiesInCoordinatesAsEvents, TemporalTracking
from babisteps.tasks.immediateorder.utils import (
    _get_list_relations,
    _get_relations_by_type,
)
from babisteps.tasks.temporaltracking.utils import task_leaf_list
from babisteps.utils import generate_framework

In [ ]:
# yaml_path
yaml_common_path = "./../babisteps/tasks/commons.yaml"
# Load the yaml file in yaml_test variable
with open(yaml_common_path, "r") as file:
    yaml_commons = yaml.safe_load(file)

In [ ]:
output_path = Path("/home/giskard/Documents/POKTscan/code/bAbI-steps/outputs")
# add output_path to yaml_commons
yaml_commons["output_path"] = output_path

In [ ]:
# Commons
num_samples = yaml_commons.get("num_samples_by_task")
num_samples = 1
total_locations = yaml_commons.get("locations")
total_actors = yaml_commons.get("actors")
total_relations = yaml_commons.get("relations")
relation_types_compatibility = yaml_commons.get("relation_types_compatibility")
# total_objects = yaml_commons.get("objects")
output_path = yaml_commons.get("output_path")
edge_qty = yaml_commons.get("edges_qty")
verbosity = "INFO"
# Task
# babisteps/tasks/temporaltracking/config.yaml
yaml_path = "./../babisteps/tasks/temporaltracking/config.yaml"
with open(yaml_path) as stream:
    try:
        yaml_cfg = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
framework = generate_framework(num_samples, task_leaf_list)
folder_name = yaml_cfg["task_name"]
# folder_path = prepare_path(output_path, folder_name)
# log_file = os.path.join(folder_path, "logs.txt")
relations_qty = yaml_cfg.get("relations_qty")
n_entities = yaml_cfg.get("entities")
n_coordinates = yaml_cfg.get("coordinates")
events_qty = yaml_cfg.get("events_qty")

r_name_by_r_type = _get_relations_by_type(total_relations)

if "edge_qty" in yaml_cfg:
    edge_qty = yaml_cfg["edge_qty"]

In [ ]:
# Temporal Order
r_type_g = "relative_event"
# Get relations compatible with the selected type
relation = _get_list_relations(
    r_type_g,
    r_name_by_r_type,
    relations_qty,
    total_relations,
    relation_types_compatibility,
)[0]

In [ ]:
shape_str = ("locations", "actors")
entities_g = np.random.choice(total_actors, size=n_entities, replace=False).tolist()
coordinates_g = np.random.choice(total_locations, size=n_coordinates, replace=False).tolist()
entities = [Entity(name=entity) for entity in entities_g]
coordinates = [Coordinate(name=coordinate) for coordinate in coordinates_g]
model = EntitiesInCoordinatesAsEvents(entities=entities, coordinates=coordinates, relation=relation)

leaf, answer, i = framework[0][0], framework[0][1], framework[0][2]
topic = leaf(answer=answer, r=relation, relation_type="relative_event")
topic

In [ ]:
path = os.path.join(os.getcwd())
log_file = os.path.join(path, "logs.txt")
runtime_name = leaf.__name__ + DELIM + str(answer) + DELIM + str(i)
gen_kwargs = {}

In [ ]:
generator = TemporalTracking(
    model=deepcopy(model)._shuffle(),
    events_qty=events_qty,
    edge_qty=edge_qty,
    topic=topic,
    verbosity=verbosity,
    log_file=log_file,
    name=runtime_name,
    **gen_kwargs if gen_kwargs is not None else {},
)
generator.generate()

In [ ]:
print(generator.topic.get_question())
print(generator.topic.answer)
for GS, r in zip(generator.graphs, ["relative_event"]):
    plt.figure(figsize=(10, 8))
    pos = nx.drawing.nx_agraph.graphviz_layout(GS.g, prog="dot")
    # Create a dictionary mapping nodes to their event attributes
    node_labels = {
        node: f"{data.get('event').entity.name}\n{data.get('event').coordinate.name}"
        for node, data in GS.g.nodes(data=True)
    }
    # Draw the graph with custom labels
    nx.draw(GS.g, pos, labels=node_labels, node_color="lightblue", node_size=1200, arrowsize=20, arrows=True)
    plt.title(r, fontsize=12)
    plt.show()
generator.get_json()